# Параллельные вычисления (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Параллельные вычисления"
* https://nalepae.github.io/pandarallel/
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_windows.ipynb
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_mac_linux.ipynb
* https://requests.readthedocs.io/en/latest/
* https://docs.python.org/3/library/pathlib.html
* https://realpython.com/python-pathlib/
* https://realpython.com/python-gil/
* https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.ThreadPool

## Задачи для совместного разбора

1. Выведите на экран слова из файла words_alpha, в которых есть две или более буквы "e" подряд.

In [2]:
#!pip install pandarallel

In [5]:
import pandas as pd

words = (
    pd.read_csv("words_alpha.txt", header=None)[0]
    .dropna()
    .sample(frac=25, replace=True)
)

In [8]:
%%file smart_f.py
import re
def f(s):
    return re.search(r"e{2,}", s) is not None

Overwriting smart_f.py


In [9]:
from smart_f import f

In [10]:
%%time
r = words[words.map(f)]

CPU times: total: 23.9 s
Wall time: 31.1 s


In [11]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [12]:
%%time
r = words[words.parallel_map(f)]

CPU times: total: 2.56 s
Wall time: 8.26 s


2. Загрузите данные о комментариях с сайта jsonplaceholder.typicode.com

![](https://i.imgur.com/AwiN8y6.png)

In [13]:
import requests
import json
from tqdm import tqdm

In [11]:
r = requests.get("https://jsonplaceholder.typicode.com/comments?postId=1")
r

<Response [200]>

In [21]:
#r.text
# r.content
#r.json()

In [17]:
from pathlib import Path

In [18]:
root = Path("posts")
root.mkdir(exist_ok=True)
f = root / "1.json"
f.name, f.stem, f.parent
list(root.iterdir())

[]

In [19]:
base_url = "https://jsonplaceholder.typicode.com/comments?postId={post_id}"

In [20]:
def process(i):
    r = requests.get(base_url.format(post_id=i))
    with open(root / f"{i}.json", "w", encoding="utf8") as fp:
        json.dump(r.json(), fp)

In [21]:
%%time
for i in tqdm(range(1, 51)):
    process(i)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.10it/s]

CPU times: total: 1.33 s
Wall time: 23.8 s


In [22]:
from multiprocessing.pool import ThreadPool

In [23]:
%%time
with ThreadPool(processes=10) as pool:
    pool.map(process, range(1, 51))

CPU times: total: 1.62 s
Wall time: 3.75 s


3. Получите множество уникальных почтовых доменов.

![](https://i.imgur.com/ceY6guU.png)

In [ ]:
data = []
for f in root.iterdir():
    with open(f, "r", encoding="utf8") as fp:
        data.append(json.load(fp))
data = [x * 20000 for x in data]

## Лабораторная работа 6

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

<p class="task" id="1"></p>

1\. Напишите функцию `f`, которая принимает на вход тэг и проверяет, удовлетворяет ли тэг следующему шаблону: `[любое число]-[любое слово]-or-less`. Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи метода _серий_ `map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

In [1]:
import re
import pandas as pd

In [2]:
%%file f_tag.py
import re
def f(tag: str) -> bool:
    pattern = r'^\d+-\w+-or-less$'
    return re.search(pattern, tag) is not None

Overwriting f_tag.py


In [2]:
tags = pd.read_csv('tag_nsteps_10m.csv')['tags'].astype(str)

In [4]:
from f_tag import f

In [5]:
%%time
tags[tags.map(f)].count()

CPU times: total: 5.48 s
Wall time: 27.4 s


288503

<!-- TODO -->

<p class="task" id="2"></p>

2\. Напишите функцию `parallel_map`, которая принимает на вход серию `s` `pd.Series` и функцию одного аргумента `f` и поэлементно применяет эту функцию к серии, распараллелив вычисления при помощи пакета `multiprocessing`. Логика работы функции `parallel_map` должна включать следующие действия:
* разбиение исходной серии на $K$ частей, где $K$ - количество ядер вашего процессора;
* параллельное применение функции `f` к каждой части при помощи метода _серии_ `map` c использованием нескольких подпроцессов;
* объединение результатов работы подпроцессов в одну серию. 

Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи `parallel_map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

In [3]:
import multiprocessing

In [4]:
import numpy as np

In [8]:
%%file f_tag2.py
import re
def f2(df: str) -> bool:
    pattern = r'^\d+-\w+-or-less$'
    return df[df.map(lambda tag: re.search(pattern, tag) is not None)]

Overwriting f_tag2.py


In [9]:
from f_tag2 import f2

In [10]:
def parallel_map(s: pd.Series, f: callable) -> pd.Series:
    K = multiprocessing.cpu_count()
    s_splitted = np.array_split(s, K)
    
    with multiprocessing.Pool(processes=K) as pool:
        r = pool.map(f, s_splitted)
    return pd.concat(r)

In [11]:
%%time
parallel_map(tags, f2).count()

CPU times: total: 531 ms
Wall time: 4.23 s


288503

<p class="task" id="3"></p>

3\. Используя пакет `pandarallel`, примените функцию `f` из задания 1 к столбцу `tags` таблицы, с которой вы работали в этом задании. Посчитайте количество тэгов, подходящих под описанный шаблон. Выведите на экран полученный результат. Измерьте время выполнения кода. 

In [12]:
from pandarallel import pandarallel

pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [13]:
%%time
tags[tags.parallel_map(f)].count()

CPU times: total: 391 ms
Wall time: 4.82 s


288503

<p class="task" id="4"></p>

4\. Сайт [DummyJSON](https://dummyjson.com/) позволяет получить набор данных о товарах в виде JSON. Воспользовавшись пакетом `requests`, получите данные о __50 товарах__ и создайте словарь, где ключом является название товара (title), а значением - список ссылок на изображения этого товара. При создании словаря замените символ `/` в названии на пробел.

Выведите на экран количество элементов полученного словаря.

In [12]:
import requests

In [13]:
url = "https://dummyjson.com/products/{prod_id}"
products = {}
for i in range(1, 51):    
    r = requests.get(url.format(prod_id=i)).json()
    products[r['title'].replace('/', ' ')] = r['images']
len(products)

50

<p class="task" id="5"></p>

5\. Напишите функцию `download_product_imgs`, которая создает папку c названием товара внутри каталога `imgs` (сам каталог `imgs` может быть создан любым удобным способом до начала работы) и сохраняет в нее изображения. Название для файла изображения извлеките из URL.

Воспользовавшись этой функцией, скачайте изображения всех продуктов. Выведите на экран общее количество загруженных файлов. Для отслеживания хода выполнения кода используйте пакет `tqdm`.

In [14]:
import json
from tqdm import tqdm
from pathlib import Path
import os
import multiprocessing

In [18]:
# пример кода для скачивания картинки
url = "https://png.pngtree.com/png-vector/20201229/ourmid/pngtree-a-british-short-blue-and-white-cat-png-image_2654518.jpg"
img = requests.get(url).content
with open("cat.jpg", "wb") as fp:
    fp.write(img)

In [23]:
root = Path(f"img")
root.mkdir(exist_ok=True)

In [19]:
def download_product_imgs(title, imgs):
    '''
    title - название товара
    imgs - список ссылок на изображения товара
    '''
    root = Path(f"img/{title}")
    root.mkdir(exist_ok=True)
    for img in imgs:
        img_content = requests.get(img).content
        img_name = img.split('/')[-1]
        with open(root/img_name, "wb") as fp:
            fp.write(img_content)
        return len(img)

In [20]:
files = 0
for k, v in tqdm(products.items()):
    files += download_product_imgs(k, v)
print(f'{files} files downloaded')

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:27<00:00,  1.74s/it]

2293 files downloaded


<p class="task" id="6"></p>

6\. Создайте функцию `download_product_imgs_processes` на основе функции `download_product_imgs`, добавив в нее вывод сообщения следующего вида: `Process ID: <ID текущего процесса>`. Для определения ID процесса воспользуйтесь функцией `multiprocessing.current_process()`.

Решите задачу 5, распараллелив вычисления при помощи процессов. Вместо корневого каталога `imgs` используйте `imgs_processes`. Выведите на экран общее количество загруженных файлов. Измерьте время выполнения кода. 

In [21]:
%%file dpi_process.py
import multiprocessing
import pathlib
import requests
from pathlib import Path

def download_product_imgs_processes(title, imgs):
    print(f"Process ID: {multiprocessing.current_process()}")
    root = Path(f"img/{title}")
    root.mkdir(exist_ok=True)
    for img in imgs:
        img_content = requests.get(img).content
        img_name = img.split('/')[-1]
        with open(root/img_name, "wb") as fp:
            fp.write(img_content)
        return len(img)

Overwriting dpi_process.py


In [22]:
from dpi_process import download_product_imgs_processes

In [23]:
%%time
K = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=K) as pool:
    res = pool.starmap(download_product_imgs_processes, zip(products.keys(), products.values()))
print(sum(res))

2293
CPU times: total: 0 ns
Wall time: 16.8 s


<p class="task" id="7"></p>

7\. Создайте функцию `download_product_imgs_threads` на основе функции `download_product_imgs`, добавив в нее вывод сообщения следующего вида: `Process ID: <ID текущего процесса> Thread ID: <ID текущего потока>`. Для определения ID потока воспользуйтесь функцией `threading.get_ident`.

Решите задачу 5, распараллелив вычисления при помощи потоков. Вместо корневого каталога `imgs` используйте `imgs_threads`. Выведите на экран общее количество загруженных файлов. Измерьте время выполнения кода. 

In [18]:
root = Path(f"imgs_threads")
root.mkdir(exist_ok=True)

In [19]:
%%file dpi_threads.py
import multiprocessing
import pathlib
import requests
import threading
from pathlib import Path

def download_product_imgs_threads(title, imgs):
    print(f"Process ID: {multiprocessing.current_process()}, Thread ID: {threading.get_ident()}")
    root = Path(f"imgs_threads/{title}")
    root.mkdir(exist_ok=True)
    for img in imgs:
        img_content = requests.get(img).content
        img_name = img.split('/')[-1]
        with open(root/img_name, "wb") as fp:
            fp.write(img_content)
        return len(img)

Overwriting dpi_threads.py


In [20]:
from dpi_threads import download_product_imgs_threads

In [21]:
import threading

In [22]:
%%time
with multiprocessing.pool.ThreadPool(processes=50) as pool:
    res = pool.starmap(download_product_imgs_threads, zip(products.keys(), products.values()))
print(sum(res))

Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 9008
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 168
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 19308
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 11316
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 3000
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 6376Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 9288

Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 14728
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 16292
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 1096
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 9028
Process ID: <_MainProcess name='MainProcess' parent=None started>, Thread ID: 2700
P

<p class="task" id="8"></p>

8\. Напишите функцию `create_2d_list`, которая создает матрицу размера `m` на `n` (__в виде списка списков__) вещественных чисел из стандартного нормального распределения. Напишите функцию `sum_by_chunk`, которая принимает на вход несколько строк этой матрицы (тоже в виде списка списков) и находит сумму элементов. 

Используя данную функцию, решите задачу поиска суммы по всей матрице тремя способами:
* передав в функцию `sum_by_chunk` всю матрицу целиком;
* распараллелив вычисления при помощи процессов по следующему принципу: матрица разбивается на части (например, по 1тыс. строк); процессы независимо друг от друга обрабатывают эти части; после завершения работы всех процессов результаты агрегируются для получения результата для всей матрицы;
* распараллелив вычисления при помощи потоков аналогичным способом.

Для демонстрации результата создайте матрицу достаточно большого размера (не менее 100 тыс. строк), выведите на экран результаты работы трех вариантов решения и измерьте время выполнения каждого из них.

В данном задании разрешается использовать пакет `numpy` только для создания матрицы. В этом случае необходимо преобразовать ее к списку списков до начала работы.

In [3]:
def create_2d_list(m, n):
    arr = np.random.standard_normal(size=(m, n)).tolist()
    return arr

In [7]:
# первый способ

In [4]:
def sum_by_chunk(matrix):
    return sum(map(sum, matrix))

In [5]:
matrix = create_2d_list(100_000, 100)

In [6]:
%%time
sum_by_chunk(matrix)

CPU times: total: 46.9 ms
Wall time: 78.1 ms


1132.4223261149227

In [8]:
# второй способ

In [9]:
%%file sbc_mul.py
def sum_by_chunk_mul(matrix):
    return sum(map(sum, matrix))

Writing sbc_mul.py


In [10]:
from sbc_mul import sum_by_chunk_mul

In [15]:
matrix = np.array(np.array_split(matrix, 1000)).tolist()

In [19]:
%%time
K = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=K) as pool:
    ans = pool.map(sum_by_chunk_mul, matrix)
sum(ans)

CPU times: total: 359 ms
Wall time: 696 ms


1132.4223261149493

In [20]:
# третий способ

In [21]:
%%time
with multiprocessing.pool.ThreadPool(processes=K) as pool:
    ans = pool.map(sum_by_chunk_mul, matrix)
sum(ans)

CPU times: total: 31.2 ms
Wall time: 109 ms


1132.4223261149493

<p class="task" id="9"></p>

9\. Напишите функцию `create_2d_arr`, которая создает матрицу размера `m` на `n` (__в виде массива numpy__) вещественных чисел из стандартного нормального распределения. Напишите функцию `sum_by_chunk_np`, которая принимает на вход несколько строк этой матрицы (тоже в виде массива) и находит сумму элементов. 

Используя данную функцию, решите задачу поиска суммы по всей матрице тремя способами аналогично задаче 8.

Для демонстрации результата создайте матрицу достаточно большого размера (не менее 100 тыс. строк), выведите на экран результаты работы трех вариантов решения и измерьте время выполнения каждого из них.

В данном задании при поиска суммы не используйте встроенную функцию `sum`, вместо этого используйте возможности `numpy`.

In [22]:
def create_2d_arr(m, n):
    arr_np = np.random.standard_normal(size=(m, n))
    return arr_np

In [34]:
%%file sbc_np.py
import numpy as np
def sum_by_chunk_np(matrix):
    return np.sum(matrix)

Writing sbc_np.py


In [24]:
matrix = create_2d_arr(100_000, 100)

In [26]:
# первый способ

In [25]:
%%time
sum_by_chunk_np(matrix)

CPU times: total: 15.6 ms
Wall time: 31.3 ms


-2777.485706043733

In [27]:
# второй способ

In [35]:
from sbc_np import sum_by_chunk_np

In [36]:
%%time
K = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=K) as pool:
    ans = pool.map(sum_by_chunk_np, matrix)
sum(ans)

CPU times: total: 406 ms
Wall time: 1.09 s


-2777.4857060437275

In [38]:
# третий способ

In [37]:
%%time
with multiprocessing.pool.ThreadPool(processes=K) as pool:
    ans = pool.map(sum_by_chunk_np, matrix)
sum(ans)

CPU times: total: 203 ms
Wall time: 437 ms


-2777.4857060437275